In [1]:
import os

In [2]:
os.chdir('../')

In [3]:
from dataclasses import dataclass
from pathlib import  Path
@dataclass(frozen=True)

class ModelConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps:int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: int
    gradient_accumulation_steps: int

In [4]:
from bpReviewClassifier.constants import *
from bpReviewClassifier.utils.common import read_yaml,create_directories

In [5]:
class ConfigurationManager:
    def __init__(self,config_filepath=CONFIG_FILE_PATH,params_filepath=PARAMS_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    def get_model_config(self)->ModelConfig:
        config=self.config.model_trainer
        params=self.params.TrainingArguments
        create_directories([config.root_dir])
        model_trainer_config=ModelConfig(
        root_dir=config.root_dir,
        data_path=config.data_path,
        model_ckpt=config.model_ckpt,
        num_train_epochs=params.num_train_epochs,
        warmup_steps=params.warmup_steps,
        per_device_train_batch_size=params.per_device_train_batch_size,
        weight_decay=params.weight_decay,
        logging_steps=params.logging_steps,
        evaluation_strategy=params.evaluation_strategy,
        eval_steps=params.eval_steps,
        save_steps=params.save_steps,
        gradient_accumulation_steps=params.gradient_accumulation_steps
        )
        return model_trainer_config



In [6]:
from transformers import  Trainer,TrainingArguments
from transformers import AutoModelForSequenceClassification,AutoTokenizer,DataCollatorWithPadding
from datasets import load_dataset,load_from_disk
import torch


c:\Users\AA\anaconda3\envs\BeautyPro\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-01-21 00:05:31,319 : INFO : config : PyTorch version 2.5.1 available.]


In [7]:
class ModelTrainer:
    def __init__(self,config:ModelConfig):
        self.config=config
    def train(self):
        device='cuda' if torch.cuda.is_available() else 'cpu'
        Tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        #data_collator=DataCollatorWithPadding(tokenizer)
        model_review_classifer=AutoModelForSequenceClassification.from_pretrained(self.config.model_ckpt,num_labels=1)
       
        transformed_ds=load_from_disk(self.config.data_path)
       
        trainer_args=TrainingArguments(
            output_dir=self.config.root_dir,
            num_train_epochs=2,
            per_device_train_batch_size=1,
            per_device_eval_batch_size=1, 
            warmup_steps=500,
            weight_decay=0.01,
            logging_steps=10,
            eval_strategy='steps',
            eval_steps=500,
            save_steps=1e6,
            gradient_accumulation_steps=16
        )
        trainer=Trainer(
            model=model_review_classifer,
            args=trainer_args,
            #data_collator=data_collator,
            tokenizer=Tokenizer,
            train_dataset=transformed_ds["test"],
            eval_dataset=transformed_ds["validation"]
        )
        trainer.train()
        model_review_classifer.save_pretrained(os.path.join(self.config.root_dir,"review-Classifier-model"))
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))
                              
        

In [ ]:
try:
    config=ConfigurationManager()
    model_trainer_config=config.get_model_config()
    model_trainer_config=ModelTrainer(model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2025-01-21 00:05:32,423 : INFO : common : yaml file config\config.yaml loaded successfully]


[2025-01-21 00:05:32,440 : INFO : common : yaml file params.yaml loaded successfully]
[2025-01-21 00:05:32,444 : INFO : common : artifacts Created successfully.]
[2025-01-21 00:05:32,447 : INFO : common : artifacts/model_training Created successfully.]


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\AA\AppData\Local\Temp\ipykernel_10904\3765089197.py:25: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer=Trainer(
c:\Users\AA\anaconda3\envs\BeautyPro\lib\site-packages\torch\nn\modules\loss.py:608: UserWarning: Using a target size (torch.Size([5])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Step,Training Loss,Validation Loss
